# Install pdbs

In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
print(sys.path[-1])

/home/cotsios/dsit/2nd-semester/algos-in-struct-bio/project/MLtopKin


In [2]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_RAW_DIR = os.path.join(BASE_DIR, "data_raw")
METADATA_DIR = os.path.join(BASE_DIR, "metadata")
DATA_DIR = os.path.join(BASE_DIR, "data")
LOGS_DIR = os.path.join(BASE_DIR, "logs")
print(BASE_DIR)
print(DATA_RAW_DIR)
print(METADATA_DIR)
print(DATA_DIR)

/home/cotsios/dsit/2nd-semester/algos-in-struct-bio/project/MLtopKin
/home/cotsios/dsit/2nd-semester/algos-in-struct-bio/project/MLtopKin/data_raw
/home/cotsios/dsit/2nd-semester/algos-in-struct-bio/project/MLtopKin/metadata
/home/cotsios/dsit/2nd-semester/algos-in-struct-bio/project/MLtopKin/data


In [3]:
metrix_db = pd.read_csv(
    os.path.join(METADATA_DIR, "metrix.csv"),
    sep=",",
    encoding="utf-8",
    header=0,
    index_col=0,
)

kincore_db = pd.read_csv(
    os.path.join(METADATA_DIR, "kincore.tab"),
    sep="\t"
)

In [4]:
kincore_db = kincore_db[kincore_db["PDB"].str.len() < 6]
kincore_db

,Organism,Group,Gene,UniprotID,PDB,Method,Resolution,Rfac,FreeRfac,DomainBoundary,...,DihedralLabel,ActivityLabel,Chelix-Saltbr,ActLoop,Ligand,LigandType,DFG_Phe,Phosphorylation,Mutations,ProteinName
0,Aplysia californica,CAMK,TWITCHIN,Q16980_APLCA,1KOBA,XRAY,2.3,0.200,0.282,47-302,...,ABAminus,Inactive,out-out,in-out,No_ligand,No_ligand,189,NaN,NaN,Twitchin-like protein
1,Aplysia californica,CAMK,TWITCHIN,Q16980_APLCA,1KOBB,XRAY,2.3,0.200,0.282,47-302,...,ABAminus,Inactive,out-out,in-out,No_ligand,No_ligand,189,NaN,NaN,Twitchin-like protein
2,Arabidopsis thaliana,CAMK,CIPK23,CIPKN_ARATH,4CZTA,XRAY,2.3,0.179,0.234,31-286,...,BLBminus,Inactive,out-out,out-out,CPS:61316,Type1,173,NaN,NaN,CBL-interacting serine/threonine-protein kinas...
3,Arabidopsis thaliana,CAMK,CIPK23,CIPKN_ARATH,4CZTB,XRAY,2.3,0.179,0.234,31-286,...,BLBminus,Inactive,out-out,out-out,CPS:61314,Type1,173,NaN,NaN,CBL-interacting serine/threonine-protein kinas...
4,Arabidopsis thaliana,CAMK,CIPK23,CIPKN_ARATH,4CZUA,XRAY,1.9,0.194,0.233,31-286,...,BLBminus,Inactive,out-out,out-out,CPS:61321,Type1,173,NaN,T190D,CBL-interacting serine/threonine-protein kinas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9692,Zea mays,CMGC,ACK2,CSK2A_MAIZE,2PVMA,XRAY,2.0,0.188,0.225,34-319,...,BLAminus,Inactive,in-out,in-in,P29:60501,Type1,171,NaN,V251A,Casein kinase II subunit alpha
9693,Zea mays,CMGC,ACK2,CSK2A_MAIZE,2PVNA,XRAY,2.0,0.256,0.308,34-319,...,BLAminus,Active,in-in,in-in,P63:60501,Type1,171,NaN,V251A,Casein kinase II subunit alpha
9694,Zea mays,CMGC,ACK2,CSK2A_MAIZE,3BE9A,XRAY,2.0,0.190,0.228,34-319,...,BLAminus,Active,in-in,in-in,P04:60501,Type1,171,NaN,V251A,Casein kinase II subunit alpha
9695,Zea mays,CMGC,ACK2,CSK2A_MAIZE,1ZOGA,XRAY,2.3,0.239,0.294,34-319,...,BLAminus,Active,in-in,in-in,K37:60501 K37:60502,Type1 Type1,171,NaN,NaN,Casein kinase II subunit alpha


In [5]:
grouped = {}
for _, row in kincore_db.iterrows():
    gene = row["Gene"]
    act_lab = row["ActivityLabel"]
    if gene not in grouped.keys():
        grouped[gene] = [act_lab]
    else:
        grouped[gene].append(act_lab)


has_both = [
    gene for gene, labels in grouped.items()
    if ('Active' in labels) and ("Inactive" in labels)
]

grouped_has_both = {
    k: v for k, v in grouped.items()
    if k in has_both
}

has_both_counts = {
    k: {"Active": v.count("Active"), "Inactive": v.count("Inactive")}
    for k, v in grouped_has_both.items()
}

active = 0
inactive = 0
for k, v in has_both_counts.items():
    ac = v["Active"]
    inac = v["Inactive"]
    print(f"{k}, active: {ac}, inactive: {inac}")
    m = min([ac, inac])
    active += m
    inactive += m


CTR1, active: 2, inactive: 1
GRK1, active: 18, inactive: 8
GRK2, active: 19, inactive: 18
GRK5, active: 3, inactive: 3
PRKACA, active: 284, inactive: 94
ROCK2, active: 6, inactive: 30
CAMKII, active: 5, inactive: 9
CDK7, active: 17, inactive: 24
CDPK1, active: 1, inactive: 12
CDPK, active: 4, inactive: 4
PTK2, active: 3, inactive: 68
SRC, active: 33, inactive: 70
AKT1, active: 19, inactive: 1
PRKCB, active: 2, inactive: 1
PRKCH, active: 2, inactive: 1
PRKCI, active: 10, inactive: 3
PRKG1, active: 9, inactive: 8
ROCK1, active: 23, inactive: 41
AURKA, active: 5, inactive: 127
CAMK1, active: 1, inactive: 1
CAMK2A, active: 32, inactive: 5
CAMK2D, active: 11, inactive: 3
CAMKK1, active: 4, inactive: 2
CASK, active: 28, inactive: 3
CHEK1, active: 157, inactive: 11
DAPK1, active: 48, inactive: 15
DAPK2, active: 35, inactive: 10
DAPK3, active: 7, inactive: 7
DCLK1, active: 9, inactive: 7
MELK, active: 25, inactive: 6
PHKG1, active: 3, inactive: 6
PIM1, active: 180, inactive: 8
PRKAA1, active: 

In [6]:
print(f"Dataset will have {active} active and {inactive} inactive conformations.")

Dataset will have 1311 active and 1311 inactive conformations.


In [7]:
pdbs_to_download = {}
for gene in has_both:
    # Get all entries for this gene
    gene_data = kincore_db[kincore_db["Gene"] == gene]
    
    # Get active and inactive PDBs
    active_pdbs = gene_data[gene_data["ActivityLabel"] == "Active"]["PDB"].tolist()
    inactive_pdbs = gene_data[gene_data["ActivityLabel"] == "Inactive"]["PDB"].tolist()
    
    # Determine how many to take from each
    m = min(len(active_pdbs), len(inactive_pdbs))
    
    # Store the PDBs
    pdbs_to_download[gene] = {
        "Active": active_pdbs[:m],
        "Inactive": inactive_pdbs[:m]
    }

pdbs_to_download

{'CTR1': {'Active': ['3PPZB'], 'Inactive': ['3P86B']},
 'GRK1': {'Active': ['4L9IB',
   '3T8OA',
   '3C4WA',
   '3C4WB',
   '3C4XA',
   '3C4XB',
   '3C4ZA',
   '3C50A'],
  'Inactive': ['3C4YA',
   '4L9IA',
   '4WBOA',
   '4WBOB',
   '4WBOC',
   '4WBOD',
   '7MT9G',
   '7MTAG']},
 'GRK2': {'Active': ['5HE0A',
   '5HE2A',
   '5HE3A',
   '5UKMA',
   '5UVCA',
   '5UKLA',
   '5HE1A',
   '6C2YA',
   '4MK0A',
   '7K7LA',
   '7K7ZA',
   '3V5WA',
   '4PNKA',
   '5WG3A',
   '5WG4A',
   '5WG5A',
   '5UUUA',
   '5UKKA'],
  'Inactive': ['1YM7B',
   '8JPBG',
   '8JPCG',
   '8JPDG',
   '1OMWA',
   '1YM7A',
   '1YM7C',
   '1YM7D',
   '2BCJA',
   '3UZSA',
   '3UZTA',
   '3PSCA',
   '3PVUA',
   '3PVWA',
   '3CIKA',
   '7PWDA',
   '3KRWA',
   '3KRXA']},
 'GRK5': {'Active': ['4TNBA', '4TNDA', '8UAPA'],
  'Inactive': ['4WNKA', '6PJXA', '8UAQA']},
 'PRKACA': {'Active': ['2VO7A',
   '2GNGA',
   '2GFCA',
   '1Q61A',
   '1SMHA',
   '1Q62A',
   '4C33A',
   '2GNFA',
   '2GNJA',
   '2GNLA',
   '2UZTA',
   '2UZUE'

In [8]:
l = [
    vv for v in pdbs_to_download.values() for vv in v.values()
]
l = [i for ii in l for i in ii]
l

['3PPZB',
 '3P86B',
 '4L9IB',
 '3T8OA',
 '3C4WA',
 '3C4WB',
 '3C4XA',
 '3C4XB',
 '3C4ZA',
 '3C50A',
 '3C4YA',
 '4L9IA',
 '4WBOA',
 '4WBOB',
 '4WBOC',
 '4WBOD',
 '7MT9G',
 '7MTAG',
 '5HE0A',
 '5HE2A',
 '5HE3A',
 '5UKMA',
 '5UVCA',
 '5UKLA',
 '5HE1A',
 '6C2YA',
 '4MK0A',
 '7K7LA',
 '7K7ZA',
 '3V5WA',
 '4PNKA',
 '5WG3A',
 '5WG4A',
 '5WG5A',
 '5UUUA',
 '5UKKA',
 '1YM7B',
 '8JPBG',
 '8JPCG',
 '8JPDG',
 '1OMWA',
 '1YM7A',
 '1YM7C',
 '1YM7D',
 '2BCJA',
 '3UZSA',
 '3UZTA',
 '3PSCA',
 '3PVUA',
 '3PVWA',
 '3CIKA',
 '7PWDA',
 '3KRWA',
 '3KRXA',
 '4TNBA',
 '4TNDA',
 '8UAPA',
 '4WNKA',
 '6PJXA',
 '8UAQA',
 '2VO7A',
 '2GNGA',
 '2GFCA',
 '1Q61A',
 '1SMHA',
 '1Q62A',
 '4C33A',
 '2GNFA',
 '2GNJA',
 '2GNLA',
 '2UZTA',
 '2UZUE',
 '2UZVA',
 '2UZWE',
 '1Q8TA',
 '1Q8WA',
 '1SVEA',
 '1SVGA',
 '1VEBA',
 '1XH4A',
 '1XH5A',
 '1XH6A',
 '1XH7A',
 '1XH8A',
 '3KKVA',
 '4IJ9A',
 '2C1AA',
 '2C1BA',
 '2F7EE',
 '2F7XE',
 '2F7ZE',
 '2JDSA',
 '2OH0E',
 '2OJFE',
 '3AG9B',
 '3ZO1A',
 '3ZO4A',
 '4IE9A',
 '5VHBA',
 '6E99A',


In [9]:
with open(os.path.join(
    METADATA_DIR,
    "pdb_ids.txt"
    ), "w") as fp:
    for i, id in enumerate(l):
        if i < len(l) - 1:
            fp.write(f"{id[:4]}, ")
        else:
            fp.write(f"{id[:4]}")